In [61]:
from brightway2 import *
import brightway2 as bw
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python
from bw2data.parameters import ActivityParameter, DatabaseParameter, ProjectParameter, Group
import sys
import math
import zipfile
from bw2data.utils import download_file

In [62]:
bw.projects.set_current('import_BDD_ecoinvent')


In [6]:
#ecoinvent
if 'ecoinvent 3.3_cutoff_ecoSpold02' in bw.databases:
    print("Database has already been imported.")
else:
    # mind that the ecoinvent file must be unzipped; then: path to the datasets subfolder
    fpei33cut = r"C:\Users\Public\Documents\ecoinvent\ecoinvent3.3\datasets"
    # the "r" makes sure that the path is read as a string - especially useful when you have spaces in your string
    ei33cut = bw.SingleOutputEcospold2Importer(fpei33cut, 'ei33')
    ei33cut
    ei33cut.apply_strategies()
    ei33cut.statistics()


Extracting XML data from 13831 datasets
Extracted 13831 datasets in 203.04 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy:

In [63]:
#write database to disk --> actually saves it
ei33cut.write_database()

Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:16:33


Title: Writing activities to SQLite3 database:
  Started: 02/26/2021 09:10:30
  Finished: 02/26/2021 09:27:03
  Total time elapsed: 00:16:33
  CPU %: 16.90
  Memory %: 10.60
Created database: ei33


Brightway2 SQLiteBackend: ei33

In [64]:
bw.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei33

In [65]:
sp = SimaProCSVImporter(r"C:\Users\TD264999\Documents\BDD_export\Test_AB\Test_AB_1\wind_turbine.csv", "eolienne")
sp.statistics()

Extracted 1 unallocated datasets in 0.03 seconds
1 datasets
39 exchanges
39 unlinked exchanges
  Type biosphere: 3 unique unlinked exchanges
  Type production: 1 unique unlinked exchanges
  Type technosphere: 35 unique unlinked exchanges


(1, 39, 39)

In [52]:
sp.apply_strategies()
sp.statistics()

Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: set_lognormal_loc_value_uncertainty_safe
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows
Applying strategy: link_iterable_by_fields
Applying s

(1, 39, 35)

In [66]:
#sp.migrate("simapro-ecoinvent-3")
sp.migrate('default-units')

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


In [67]:
sp.strategies[:5]

[<function bw2io.strategies.generic.normalize_units>,
 <function bw2io.strategies.locations.update_ecoinvent_locations>,
 <function bw2io.strategies.generic.assign_only_product_as_production>,
 <function bw2io.strategies.biosphere.drop_unspecified_subcategories>,
 <function bw2io.strategies.simapro.sp_allocate_products>]

In [55]:
sp.apply_strategies(sp.strategies[:5])


Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applied 5 strategies in 0.00 seconds


In [81]:
from bw2io import *
sp.apply_strategy(functools.partial(
    link_iterable_by_fields, 
    other=Database("ei33"),
    kind="technosphere",
    fields=["reference product", "name", "unit", "location"]
))

sp.statistics()

Applying strategy: link_iterable_by_fields
1 datasets
39 exchanges
39 unlinked exchanges
  Type biosphere: 3 unique unlinked exchanges
  Type production: 1 unique unlinked exchanges
  Type technosphere: 35 unique unlinked exchanges


(1, 39, 39)

In [56]:
#from bw2io.utils import activity_hash

#def match_by_activity_hash(data):
#    eidb33 = Database("ei33")
#    fields = ["name", "unit", "location"]
 #   eidb33_hashes = {activity_hash(ds, fields=fields) for ds in eidb33}
  #  #assert len(eidb33) == len(eidb33_hashes)
   # return [obj for obj in data if activity_hash(obj, fields=fields) not in eidb33_hashes]

In [57]:
#sp.apply_strategy(match_by_activity_hash)
#sp.statistics()

Applying strategy: match_by_activity_hash
1 datasets
39 exchanges
35 unlinked exchanges
  Type technosphere: 35 unique unlinked exchanges


(1, 39, 35)

In [80]:
sp.match_database("ei33", ignore_categories=True)
sp.statistics()

Applying strategy: link_iterable_by_fields
1 datasets
39 exchanges
39 unlinked exchanges
  Type biosphere: 3 unique unlinked exchanges
  Type production: 1 unique unlinked exchanges
  Type technosphere: 35 unique unlinked exchanges


(1, 39, 39)

In [75]:
for i, e in enumerate(sp.unlinked):
    print(e['name'], e['unit'], e['categories'])
    if i > 40:
        break

Wind turbine, 4.5MW, onshore {GLO}| construction | SE p ('Electricity by fuel', 'Wind', 'Transformation', 'Infrastructure')
Occupation, industrial area m2a ('Resources', 'land')
Transformation, to industrial area m2 ('Resources', 'land')
Transformation, from pasture, man made m2 ('Resources', 'land')
Excavation, hydraulic digger {GLO}| market for | Alloc Def, U m3 ('Materials/fuels',)
Steel, chromium steel 18/8 {GLO}| market for | Alloc Def, U kg ('Materials/fuels',)
Polyethylene, high density, granulate {GLO}| market for | Alloc Def, U kg ('Materials/fuels',)
Polyvinylchloride, emulsion polymerised {GLO}| market for | Alloc Def, U kg ('Materials/fuels',)
Aluminium, wrought alloy {GLO}| market for | Alloc Def, U kg ('Materials/fuels',)
Glass fibre reinforced plastic, polyamide, injection moulded {GLO}| market for | Alloc Def, U kg ('Materials/fuels',)
Sheet rolling, steel {GLO}| market for | Alloc Def, U kg ('Materials/fuels',)
Sheet rolling, chromium steel {GLO}| market for | Alloc De

In [76]:
sp.write_excel()

Wrote matching file to:
C:\Users\TD264999\AppData\Local\pylca\Brightway3\import_BDD_ecoinvent.b8499629751d715829a9e9cc3bdb3ffe\output\db-matching-eolienne.xlsx
